In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from nltk.probability import FreqDist
import nltk
import numpy as np
import warnings
warnings.filterwarnings("ignore")
# 测试样本数
N = 90


def load_user_cmd_new(filename):
    cmd_list = []
    dist = []
    with open(filename) as f:
        i = 0
        x = []
        for line in f:
            line = line.strip('\n')
            x.append(line)
            dist.append(line)
            i += 1
            if i == 100:
                cmd_list.append(x)
                x = []
                i = 0

    fdist = FreqDist(dist)  # 按照频率排序
    # for i in fdist:
    #     print(i, fdist[i])
    return cmd_list, fdist


def load_user_cmd(filename):
    cmd_list = []
    dist_max = []
    dist_min = []
    dist = []
    with open(filename) as f:
        i = 0
        x = []
        for line in f:
            line = line.strip('\n')
            x.append(line)
            dist.append(line)
            i += 1
            if i == 100:
                cmd_list.append(x)
                x = []
                i = 0

    fdist = FreqDist(dist).keys()
    dist_max = set(fdist[0:50])
    dist_min = set(fdist[-50:])
    return cmd_list, dist_max, dist_min


def get_user_cmd_feature(user_cmd_list, dist_max, dist_min):
    user_cmd_feature = []
    for cmd_block in user_cmd_list:
        f1 = len(set(cmd_block))
        fdist = FreqDist(cmd_block).keys()
        f2 = fdist[0:10]
        f3 = fdist[-10:]
        f2 = len(set(f2) & set(dist_max))
        f3 = len(set(f3) & set(dist_min))
        x = [f1, f2, f3]
        user_cmd_feature.append(x)
    return user_cmd_feature


def get_user_cmd_feature_new(user_cmd_list, dist):
    user_cmd_feature = []

    for cmd_list in user_cmd_list:
        v = [0]*len(dist)
        for i in range(0, len(dist)):
            if dist[i] in cmd_list:
                v[i] += 1
        user_cmd_feature.append(v)

    return user_cmd_feature


def get_label(filename, index=0):
    x = []
    with open(filename) as f:
        for line in f:
            line = line.strip('\n')
            x.append(int(line.split()[index]))
    return x



In [4]:
user_cmd_list, dist = load_user_cmd_new("../../Data/MasqueradeDat/User3")
# user_cmd_list:一共150组,每组100个
# dist: 107个,按照频率降序排列
print(len(dist))
print( "dist:%s" % dist)
for i in dist:
    print(i,end=' ')
    
print()
# user_cmd_feature共150组，每组107个0
user_cmd_feature = get_user_cmd_feature_new(user_cmd_list, dist)
# labels为label文件中的第三列所有
labels = get_label("../../Data/MasqueradeDat/label.txt", 2)
y = [0]*50+labels #在labels前加了50个0

x_train = user_cmd_feature[0:N]
y_train = y[0:N]

x_test = user_cmd_feature[N:150]
y_test = y[N:150]

neigh = KNeighborsClassifier(n_neighbors=3)
# 使用x_train为训练数据，y_train为目标值
neigh.fit(x_train, y_train)
# predict函数为预测函数
y_predict = neigh.predict(x_test)
# cross_val_score:通过交叉验证评估分数。
a=cross_val_score(neigh, user_cmd_feature, y, n_jobs=-1, cv=10)
print(a)
print(np.mean(a))


107
dist:<FreqDist with 107 samples and 15000 outcomes>
expr egrep java dirname ls basename .java_wr rm make uname ex javac diff sh ksh get netscape stty sccs ln grep generic sed hostname date more xterm cat pq cfe xmodmap driver ugen as1 true a.out ld_ mailx sendmail tcpostio Xsession bdiff xsetroot tail find chmod lp mkdir FIFO readacct runnit delta id nawk getopt LOCK wc cpp xrdb FvwmPage GoodStuf fvwm xdm admin netstat post tracerou man gethost download dbx dbxpcs seecalls tcppost file awk getpgrp MediaMai rmdir gs touch bc UNLOCK ppost unpack col postprin cpio nslookup rlogin ptelnet logname launchef sim301bS ps od pr xargs gzip jar xhost hpost telnet tset matlab mimencod sim301bK 
[0.93333333 0.93333333 0.93333333 0.93333333 0.93333333 0.93333333
 0.93333333 0.93333333 0.93333333 0.86666667]
0.9266666666666667
